In [5]:
########## LagLlama ZeroShot #################

import os
import torch
import numpy as np
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation import make_evaluation_predictions
from lag_llama.gluon.estimator import LagLlamaEstimator

# Function to get LagLlama predictions
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=60, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]
    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(2.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }
    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,
        batch_size=60,
        num_parallel_samples=100,
        device=device,
    )
    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)
    return forecasts, tss


horizon = 12
products = sorted([name for name in os.listdir('../uf/') if os.path.isdir(os.path.join('../uf/', name))])
all_forecasts = []

# Loop to process and combine all forecasts
for product in products:
    folder_path = f'../uf/{product}/'
    files = os.listdir(folder_path)

    for file_name in files:
        if file_name.endswith('.csv') and 'mensal' in file_name:
            estado = file_name.split('_')[1]  # Extract state from file name
            print(f"Processing Product: {product}, State: {estado}")

            flattened_forecasts = []
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path, sep=";")
            df = df.iloc[:-horizon]  # Exclude the last 'horizon' rows
            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m')
            df = df.rename(columns={'m3': 'target'})
            df.set_index('timestamp', inplace=True)
            df['item_id'] = f"{product}_{estado}"
            df['target'] = df['target'].astype('float32')

            dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            forecasts, tss = get_lag_llama_predictions(dataset, prediction_length=horizon, device=device)

            for forecast in forecasts:
                year_forecast = forecast.samples.mean(axis=0)
                flattened_forecasts.extend(year_forecast[:horizon])

            row_with_metadata = [product, estado] + flattened_forecasts
            all_forecasts.append(row_with_metadata)

# Save all forecasts to a single CSV file
output_file = "all_forecasts_combined.csv"
columns = ["Product", "Estado"] + [f"Point_{i+1}" for i in range(horizon)]
pd.DataFrame(all_forecasts, columns=columns).to_csv(output_file, index=False)

print(f"Saved all forecasts to {output_file}")


Processing Product: etanolhidratado, State: rs
Processing Product: etanolhidratado, State: pr


/tmp/ipykernel_3703416/1288246799.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device)
/home/user/miniconda3/envs/ra

Processing Product: etanolhidratado, State: ms
Processing Product: etanolhidratado, State: sc


/tmp/ipykernel_3703416/1288246799.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device)
/home/user/miniconda3/envs/ra

Processing Product: etanolhidratado, State: ba
Processing Product: etanolhidratado, State: pa


...

Processing Product: queroseneiluminante, State: rr
Processing Product: queroseneiluminante, State: pa


/tmp/ipykernel_3703416/1288246799.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device)
/home/user/miniconda3/envs/ra

Processing Product: queroseneiluminante, State: to
Processing Product: queroseneiluminante, State: pr


/tmp/ipykernel_3703416/1288246799.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device)
/home/user/miniconda3/envs/ra

Saved all forecasts to all_forecasts_combined.csv
